# Importing Libraries and Loading datasets

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from joblib import dump, load
import matplotlib.pyplot as plt

!git clone https://github.com/analokmaus/kuma_utils.git
import sys; sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression

# Load data

In [14]:
train = pd.read_csv("../input/tabular-playground-series-aug-2022/train.csv", index_col='id')
test = pd.read_csv("../input/tabular-playground-series-aug-2022/test.csv", index_col='id')
sub = pd.read_csv("../input/tabular-playground-series-aug-2022/sample_submission.csv")

#train.head()
#train.describe()

train_y = train.failure.copy()
train = train.drop('failure', axis=1)
print('Train data shape:', train.shape)
print('Test data shape:', test.shape)
print('Train data failure shape:', train_y.shape)

Train data shape: (26570, 24)
Test data shape: (20775, 24)
Train data failure shape: (26570,)


# Numerical & Categorical Features 

In [15]:
# eliminate non-numerical data(product_code, attribute_0, attribute_1):categorical features
# column number: 24 -> 21
numerical_cols = train.select_dtypes(np.number).columns.values.tolist()
categorical_cols = [x for x in train.columns.values if (x not in numerical_cols)]

# Fill Missing Values

In [16]:
# calculate missing number of dataset
missing_values_train = train.isna().sum().sum()
print('Missing values in train data: {0}'.format(missing_values_train))
missing_values_test = test.isna().sum().sum()
print('Missing values in test data: {0}\n'.format(missing_values_test))

# from kuma_utils
imputer = LGBMImputer(n_iter=100)
imputer.fit(train[numerical_cols].append(test[numerical_cols]))
train[numerical_cols] = imputer.transform(train[numerical_cols])
test[numerical_cols] = imputer.transform(test[numerical_cols])

missing_values_train = train.isna().any().sum()
print('\nMissing values in train data: {0}'.format(missing_values_train))
missing_values_test = test.isna().any().sum()
print('Missing values in test data: {0}'.format(missing_values_train))

Missing values in train data: 20273
Missing values in test data: 15709



  0%|          | 0/16 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]


Missing values in train data: 0
Missing values in test data: 0


# Feature Engineering

In [17]:
def feature_engineering(data, numerical_cols):
    meas_gr1_cols = [f"measurement_{i:d}" for i in list(range(3, 5)) + list(range(9, 18))] 
    meas_gr2_cols = [f"measurement_{i:d}" for i in list(range(5, 9))]
    
    data['attribute_2*3'] = data['attribute_2'] * data['attribute_3']
    data['meas_gr1_avg'] = np.mean(data[meas_gr1_cols], axis=1)
    data['meas_gr1_std'] = np.std(data[meas_gr1_cols], axis=1)
    data['meas_gr2_avg'] = np.mean(data[meas_gr2_cols], axis=1)
    data['meas17/meas_gr2_avg'] = data['measurement_17'] / data['meas_gr2_avg']

    numerical_cols = numerical_cols + ['attribute_2*3']
    numerical_cols = numerical_cols + ['meas_gr1_avg']
    numerical_cols = numerical_cols + ['meas_gr1_std']
    numerical_cols = numerical_cols + ['meas_gr2_avg']   
    numerical_cols = numerical_cols + ['meas17/meas_gr2_avg']
    
    for column in categorical_cols:
        # from sklearn
        label_encoder = LabelEncoder()
        label_encoder.fit(train[column].append(test[column]))
        train[column] = label_encoder.transform(train[column])
        test[column] = label_encoder.transform(test[column])    
    
    #data['loading'] = np.log1p(data['loading'])
    return numerical_cols

feature_engineering(train, numerical_cols)
numerical_cols = feature_engineering(test, numerical_cols)
train.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,attribute_2*3,meas_gr1_avg,meas_gr1_std,meas_gr2_avg,meas17/meas_gr2_avg
id,,,,,,,,,,,,,,,,,,,,,
0,0,80.10,1,3,9,5,7,8,4,18.040,...,15.029,15.400725,13.034,14.684,764.100,45,82.920339,215.417497,16.73350,45.662892
1,0,84.89,1,3,9,5,14,3,3,18.213,...,14.732,15.425000,14.395,15.631,682.057,45,75.641636,191.779013,16.56175,41.182665
2,0,82.43,1,3,9,5,12,1,5,18.057,...,16.711,18.631000,14.094,17.946,663.376,45,74.660690,186.182903,16.49600,40.214355
3,0,101.07,1,3,9,5,13,2,6,17.295,...,15.250,15.562000,16.154,17.172,826.282,45,88.737909,233.245686,17.13950,48.209224
4,0,188.06,1,3,9,5,9,2,8,19.346,...,16.182,12.760000,13.153,16.412,579.885,45,66.406091,162.402011,15.53375,37.330651


# Modelling

In [20]:
# features selected from "features selection" above
select_feature = ['attribute_0', 'meas_gr2_avg', 'measurement_1', 'measurement_17', 'measurement_12', 'meas_gr1_std', 'attribute_2', 'attribute_1', 'measurement_2', 'measurement_7', 'measurement_3', 'loading']
select_feature

['attribute_0',
 'meas_gr2_avg',
 'measurement_1',
 'measurement_17',
 'measurement_12',
 'meas_gr1_std',
 'attribute_2',
 'attribute_1',
 'measurement_2',
 'measurement_7',
 'measurement_3',
 'loading']

# Load Model & Predict

In [24]:
model = load('model.joblib') 
predictions = model.predict_proba(test[select_feature])[:, 1]

print(predictions)
sub['failure'] = predictions
sub[['id', 'failure']].to_csv('109550111.csv', index=False)
sub.head()

[0.19021271 0.16378102 0.1770735  ... 0.14123784 0.21167554 0.16045033]


,id,failure
0,26570,0.190213
1,26571,0.163781
2,26572,0.177074
3,26573,0.180766
4,26574,0.336974
